In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
# from utils.df_handle import *
# import pendulum
# from airflow import DAG
# from airflow.operators.dummy_operator import DummyOperator
# from airflow.operators.python_operator import PythonOperator

# local_tz = pendulum.timezone("Asia/Bangkok")

# name='ERR'
# prefix='SYNC_'
# csv_path = '/usr/local/airflow/plugins'+'/'

# dag_params = {
#     'owner': 'airflow',
#     "depends_on_past": False,
#     'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
#     'email_on_failure': True,
#     'email_on_retry': False,
#     'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
#     'do_xcom_push': False,
#     'execution_timeout':timedelta(seconds=300)
# }

# dag = DAG(prefix+name,
#           catchup=False,
#           default_args=dag_params,
#           schedule_interval= '*/30 8-17,23-23 * * *',
#           tags=[prefix+name, 'Sync', '30mins']
# )

In [2]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")
datenow_mns45 = (datetime.now() - timedelta(days=45)).strftime("%Y-%m-%d")

In [5]:
from_tb = "API_HistoryOM205"

In [4]:
table_name = "sync_dms_err"
table_temp = "sync_dms_err_temp"

In [17]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-01'
DECLARE @StringApproveOrderT1 INT = ( SELECT LEN(N'Đơn Hàng DH032021-00257 Tồn Tại Các Lỗi:</br>') + 1)
select
CONCAT(BranchID,OrderNbr) as pk,
BranchID,
OrderNbr,
DateImport as Crtd_DateTime,
DateUpdate as LUpd_DateTime,
REPLACE(
REPLACE
(SUBSTRING(ErrorMessage, @StringApproveOrderT1, LEN(ErrorMessage)),
N'</br>Bạn Không Thể Duyệt Đơn.',
''),
'</br>',
', '
)
AS ErrorMessage
from {from_tb} 
where Status = 'E'
and DateUpdate >= @from
"""

In [18]:
print(usql)


DECLARE @from DATE = '2022-05-15'
DECLARE @to DATE = '2022-01-01'
DECLARE @StringApproveOrderT1 INT = ( SELECT LEN(N'Đơn Hàng DH032021-00257 Tồn Tại Các Lỗi:</br>') + 1)
select
CONCAT(BranchID,OrderNbr) as pk,
BranchID,
OrderNbr,
DateImport as Crtd_DateTime,
DateUpdate as LUpd_DateTime,
REPLACE(
REPLACE
(SUBSTRING(ErrorMessage, @StringApproveOrderT1, LEN(ErrorMessage)),
N'</br>Bạn Không Thể Duyệt Đơn.',
''),
'</br>',
', '
)
AS ErrorMessage
from API_HistoryOM205 
where Status = 'E'
and DateUpdate >= @from



In [19]:
def update():
    try:
        #UPDATE
        df_update = get_ms_df(usql)
        assert df_update.shape[0] >0,"NO DATA TO INPUT"
        df_update.columns = lower_col(df_update)
        # df_update.columns
        df_update['crtd_datetime'] = df_update.crtd_datetime.dt.normalize()
        df_update1 = df_update['crtd_datetime']
        df_update1.drop_duplicates(inplace=True)
        tpl_dt = tuple(df_update1.dt.strftime('%Y-%m-%d').to_list()) + ('1900-01-01',)
        # tpl_dt
        df_update1 = df_update['pk']
        df_update1.drop_duplicates(inplace=True)
        tpl_pk = tuple(df_update1.to_list()) + ('',)
        del_sql = \
        f"""
        DELETE FROM biteam.{table_name}
        WHERE
        DATE(crtd_datetime) in {tpl_dt}
        AND pk in {tpl_pk}
        """
        print("del_sql ",del_sql)
        execute_bq_query(del_sql)
        df_update['inserted_at'] = datetime.now()
        bq_values_insert(df_update, f"{table_name}", 2)
    except AssertionError:
        print("NO DATA TO INPUT")

In [20]:
# update()

del_sql  
        DELETE FROM biteam.sync_dms_err
        WHERE
        DATE(crtd_datetime) in ('2022-05-15', '1900-01-01')
        AND pk in ('MR0001DH0-0422-02105', 'MR0001DH0-0522-01350', 'MR0001DH0-0522-01411', 'MR0001DH0-0522-01774', 'MR0001DH0-0522-01801', 'MR0001DH0-0522-02120', 'MR0001DH0-0522-01995', 'MR0001DH0-0522-02281', 'MR0010DH1-0522-01171', 'MR0010DH1-0522-01203', 'MR0010DH1-0522-01204', 'MR0010DH1-0522-01304', 'MR0010DH1-0522-01555', 'MR0010DH1-0522-01469', 'MR0010DH1-0522-01617', 'MR0010DH1-0522-01653', 'MR0010DH1-0522-01709', 'MR0010DH1-0522-01711', 'MR0010DH1-0522-01732', 'MR0010DH1-0522-01742', 'MR0010DH1-0522-01749', 'MR0012DH2-0522-00450', 'MR0012DH2-0522-00455', 'MR0012DH2-0522-00465', 'MR0012DH2-0522-00487', 'MR0012DH2-0522-00502', 'MR0012DH2-0522-00505', 'MR0012DH2-0522-00507', 'MR0012DH2-0522-00513', 'MR0012DH2-0522-00516', 'MR0012DH2-0522-00517', 'MR0013DH3-0522-00581', 'MR0013DH3-0522-00649', 'MR0013DH3-0522-00656', 'MR0013DH3-0522-00726', 'MR0013DH3-0522-0

1it [00:03,  3.30s/it]


In [ ]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> update